In [ ]:
# Install the matplotlib library
!pip install matplotlib


In [ ]:
# Import necessary libraries
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import uuid


In [ ]:
# Setup paths for storing data
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

# Make directories if they don't exist
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

# Generate unique image name using UUID library
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))


In [ ]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)


In [ ]:
# Start a loop to collect images from webcam
while cap.isOpened():
    ret, frame = cap.read()

    # Crop frame to 250x250 pixels
    frame = frame[120:120+250,200:200+250, :]

    # Collect anchor images on 'a' keypress
    if cv2.waitKey(1) & 0XFF == ord('a'):
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)

    # Collect positive images on 'p' keypress
    if cv2.waitKey(1) & 0XFF == ord('p'):
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)

    # Show image
    cv2.imshow('Image Collection', frame)

    # Break loop on 'q' keypress
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

# Release webcam and close image window
cap.release()
cv2.destroyAllWindows()


In [ ]:
# Load anchor, positive, and negative images into datasets
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(35)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(35)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(35)


In [ ]:
# Test dataset
dir_test = anchor.as_numpy_iterator()
print(dir_test.next())


In [ ]:
# Preprocess images
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img,(100,100))
    img = img/255.0
    return img


In [ ]:
img = preprocess('data\\anchor\\1a0464d9-7e22-11ee-a4d7-00933761849b.jpg')

plt.imshow(img)


In [ ]:
# Create labels for anchor-positive and anchor-negative pairs
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)


In [ ]:
# Labels for classification
class_labels = tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))
iterator_labs = class_labels.as_numpy_iterator()
iterator_labs.next()


In [ ]:
# Preprocess images for training
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)


In [ ]:
res = preprocess_twin(*examples)

plt.imshow(res[1])


In [ ]:
# Build data loader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

samples = data.as_numpy_iterator()
samples.next()


In [ ]:
# Divide data into train and test partitions
train_data = data.take(round(len(data)*0.7))
train_data = train_data.batch(8)
train_data = train_data.prefetch(4)

test_data = data.skip(round(len(data)*0.7))
test_data = test_data.take(round(len(data) *0.3))
test_data = test_data.batch(8)
test_data = test_data.prefetch(4)


In [ ]:
# Define the architecture of the embedding model
inp = Input(shape=(100,100,3), name='input_image')
c1 = Conv2D(64, (10,10), activation='relu')(inp)
m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
c2 = Conv2D(128, (7,7), activation='relu')(m1)
m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
c3 = Conv2D(128, (4,4), activation='relu')(m2)
m3 = MaxPooling2D(64
